In [16]:
GOOGLE_API_KEY=secret

In [9]:
!pip install flask flask-ngrok
!pip install PyPDF2 langchain langchain-community langchain-google-genai
!pip install chromadb sentence-transformers google-generativeai
!pip install python-dotenv numpy tiktoken
!pip install pyngrok==4.1.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 18.2 MB/s eta 0:

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=d3027ee06b046257cbd6b77bdd81c5f822433b3b168f2c0d0ef9b4c6b3ccd1fd
  Stored in directory: /root/.cache/pip/wheels/f4/e2/28/275496a22cb15fb495458aed2b164b4ccaa10e2f44188fb0b9
Successfully built pyngrok


In [10]:
!ngrok authtoken YOUR_NGROK_AUTH_TOKEN_HERE

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [17]:
import os
import json
import logging
from typing import List, Dict, Optional
import google.generativeai as genai
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
import uuid



In [17]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class RAGQuestionAnsweringSystem:
    def __init__(self, google_api_key: str):
        self.google_api_key = google_api_key
        genai.configure(api_key=google_api_key)

        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        self.llm = ChatGoogleGenerativeAI(
            model="gemini-pro",
            google_api_key=google_api_key,
            temperature=0.3
        )

        self.vector_stores = {}
        self.documents = {}
        self.questions = {}
        self.session_metadata = {}


In [ ]:
def create_session(self) -> str:
        session_id = str(uuid.uuid4())
        self.session_metadata[session_id] = {
            "created_at": None,
            "pdf_name": None,
            "processed": False
        }
        return session_id

    def process_pdf(self, pdf_path: str, session_id: str, pdf_name: str = None) -> bool:
        try:
            loader = PyPDFLoader(pdf_path)
            documents = loader.load()

            if not documents:
                logger.error("Empty PDF")
                return False

            splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200,
                length_function=len,
                separators=["\n\n", "\n", " ", ""]
            )
            chunks = splitter.split_documents(documents)

            if not chunks:
                logger.error("No text chunks from PDF")
                return False

            persist_directory = f"./chroma_db_{session_id}"
            vector_store = Chroma.from_documents(
                documents=chunks,
                embedding=self.embeddings,
                persist_directory=persist_directory
            )

            self.vector_stores[session_id] = vector_store
            self.documents[session_id] = chunks
            self.session_metadata[session_id].update({
                "pdf_name": pdf_name,
                "processed": True,
                "chunk_count": len(chunks)
            })

            logger.info(f"{len(chunks)} chunks processed for session {session_id}")
            return True

        except Exception as e:
            logger.error(f"process_pdf error: {str(e)}")
            return False

In [ ]:
def generate_questions(self, session_id: str, num_questions: int = 5) -> List[str]:
        if session_id not in self.documents:
            logger.error(f"No documents for session {session_id}")
            return []

        try:
            sample_content = "\n".join([
                doc.page_content[:500]
                for doc in self.documents[session_id][:min(5, len(self.documents[session_id]))]
            ])

            prompt = f"""
            Based on the following document content, generate {num_questions} diverse and relevant questions.

            Questions should cover:
            1. Main concepts
            2. Specific details
            3. Interpretation
            4. Application

            Document:
            {sample_content}

            Format:
            {{"questions": ["Question 1?", "Question 2?", "Question 3?"]}}
            """

            model = genai.GenerativeModel('gemini-pro')
            response = model.generate_content(prompt)
            response_text = response.text
            questions = []

            if "{" in response_text and "}" in response_text:
                try:
                    start = response_text.find("{")
                    end = response_text.rfind("}") + 1
                    json_str = response_text[start:end]
                    parsed = json.loads(json_str)
                    questions = parsed.get("questions", [])
                except json.JSONDecodeError:
                    lines = response_text.split('\n')
                    questions = [line.strip() for line in lines if line.strip().endswith('?')][:num_questions]
            else:
                lines = response_text.split('\n')
                questions = [line.strip() for line in lines if line.strip().endswith('?')][:num_questions]

            questions = questions[:num_questions]
            self.questions[session_id] = questions

            logger.info(f"{len(questions)} questions generated for session {session_id}")
            return questions

        except Exception as e:
            logger.error(f"generate_questions error: {str(e)}")
            return []

    def get_answer(self, session_id: str, question: str) -> Dict:
        if session_id not in self.vector_stores:
            return {"error": "No processed document for session"}

        try:
            retriever = self.vector_stores[session_id].as_retriever(
                search_kwargs={"k": 3}
            )
            qa_chain = RetrievalQA.from_chain_type(
                llm=self.llm,
                chain_type="stuff",
                retriever=retriever,
                return_source_documents=True
            )
            result = qa_chain({"query": question})
            answer = result["result"]
            sources = result["source_documents"]

            return {
                "answer": answer,
                "sources": [
                    {
                        "page": doc.metadata.get("page", "Unknown"),
                        "content": doc.page_content[:200] + "..."
                    } for doc in sources
                ],
                "confidence": "high" if len(sources) >= 2 else "medium"
            }

        except Exception as e:
            logger.error(f"get_answer error: {str(e)}")
            return {"error": f"Answer generation error: {str(e)}"}

    def get_session_info(self, session_id: str) -> Optional[Dict]:
        return self.session_metadata.get(session_id)

    def get_questions_for_session(self, session_id: str) -> List[str]:
        return self.questions.get(session_id, [])

    def cleanup_session(self, session_id: str) -> bool:
        try:
            self.vector_stores.pop(session_id, None)
            self.documents.pop(session_id, None)
            self.questions.pop(session_id, None)
            self.session_metadata.pop(session_id, None)

            path = f"./chroma_db_{session_id}"
            if os.path.exists(path):
                import shutil
                shutil.rmtree(path)

            logger.info(f"Session {session_id} cleaned up")
            return True

        except Exception as e:
            logger.error(f"cleanup_session error: {str(e)}")
            return False

    def list_active_sessions(self) -> List[str]:
        return list(self.session_metadata.keys())